In [1]:
import ipywidgets as widgets
import torch
import PIL
from IPython.display import display
import io
import numpy as np
import bqplot
from matplotlib import pyplot as plt
import matplotlib as mpl

from ppuu.eval_mpc_visualizer import EvalVisualizer

class WidgetEvalVisualizer(EvalVisualizer, widgets.VBox):
    def __init__(self, output_dir=None):
        EvalVisualizer.__init__(self, output_dir, notebook=True)
        widgets.VBox.__init__(self, [self.costs_plot_output])
        
    def setup_mpl(self):
        # in the other class we'd do use non-visual backend
        pass
    
    def show_mpl(self):
        plt.show()

visualizer = WidgetEvalVisualizer()
display(visualizer)

WidgetEvalVisualizer(children=(Output(),))

In [2]:
import ppuu.eval_mpc_fm 
from ppuu.costs.policy_costs_km import PolicyCostKMTaper
import imp
imp.reload(ppuu.eval_mpc_fm)

config = ppuu.eval_mpc_fm.EvalMPCConfig.parse_from_flat_dict(
    dict(
    lambda_p=1.0, lambda_o=1.0, lambda_l=0, lr=0.10, u_reg=0.2, lambda_a=0, lambda_j=0,
    skip_contours=True, masks_power_x=9, masks_power_y=9, timestep=0.1, safe_factor=1.5, unfold_len=30, n_iter=5,
    dataset="/home/us441/nvidia-collab/vlad/traffic-data-5/state-action-cost/data_i80_v0/",
    forward_model_path="/home/us441/nvidia-collab/vlad/results/fm/km_no_action/fm_km_no_action_64/seed=42/checkpoints/last.ckpt",
    visualizer=visualizer,
    num_processes=0,
    test_size_cap=1,
    optimizer='Adam',
    fm_unfold_samples=1,
    fm_unfold_samples_agg='keep',
    planning_freq=1,
    lambda_j_mpc=0.000,
    batch_size=1,
    mask_coeff=10.0,
    update_ref_period=100.0,
    lambda_r=1.0,
    lambda_d=0.01,
    pass_gt_future=False,
    #output_dir='./output_feb17_2/',
    dataset_partition='test',
    save_opt_stats=True,
    cost_type='continuous',
    )
)
# policy = ppuu.eval_mpc.main(config)

In [ ]:
## import cProfile
# cProfile.run('ppuu.eval_mpc.main(config)', 'restats')
ppuu.eval_mpc_fm.main(config)

In [4]:
example = torch.load('bad_example.dump')

In [4]:
example['images']

NameError: name 'example' is not defined

In [ ]:
import numpy as np
values = np.arange(-3, 3, 0.1)
results = []
for v in values:
    metadata = {}
    policy(example['images'], example['states'], True, True, example['car_size'], init=torch.tensor([v, 0.0]), metadata=metadata)
    results.append(metadata['cost'])

In [ ]:
class EpisodeReview(widgets.VBox):
    def __init__(self, frames):
        self.frames = frames
        self.episode_play = widgets.Play(
            value=0,
            min=0,
            max=len(frames),
            step=1,
            description="Press play",
            disabled=False,
            interval=10,
        )
        self.update_interval_slider = widgets.IntSlider(
            min=1, max=300, value=30,
        )

        self.episode_image = widgets.Image(
            format="png", width=600, height=600,
        )
        self.episode_slider = widgets.IntSlider()
        widgets.jslink(
            (self.episode_play, "value"), (self.episode_slider, "value")
        )
        widgets.jslink(
            (self.episode_play, "max"), (self.episode_slider, "max")
        )
        widgets.jslink(
            (self.episode_play, "min"), (self.episode_slider, "min")
        )
        
        widgets.jslink(
            (self.update_interval_slider, "value"),
            (self.episode_play, "interval"),
        )

        
        def episode_slider_callback(change):
            if change.name == "value" and change.new is not None:
                self.episode_image.value = self.frames[change.new]

        self.episode_slider.observe(episode_slider_callback, type="change")
        
        super().__init__(
            [self.episode_play, self.episode_slider, self.update_interval_slider, self.episode_image]
        )

In [ ]:
episode_view = EpisodeReview(visualizer.images_history[0])
display(episode_view)

In [ ]:
import pstats
s = pstats.Stats('restats')

In [ ]:
s.strip_dirs().sort_stats(pstats.SortKey.CUMULATIVE).print_stats()

In [ ]:
import torch

from ppuu.modeling.km import predict_states, predict_states_seq
from ppuu.data.dataloader import Normalizer

bsize = 1
n_actions = 10

states = torch.randn(bsize, 5)
actions = torch.randn(bsize, n_actions, 2)

normalizer = Normalizer.dummy()

def naive(states, actions, normalizer):
    predictions = []
    for i in range(actions.shape[1]):
        states = predict_states(
            states,
            actions[:, i],
            normalizer,
        )
        predictions.append(states)
    return torch.stack(predictions, dim=1)

s1 = naive(states, actions, normalizer)
s2 = predict_states_seq(states, actions, normalizer)

In [ ]:
s1.shape

In [ ]:
s2.shape

In [ ]:
(s1 - s2).abs().max()

In [ ]:
! ls output_feb17_2/episode_data

In [ ]:
x = torch.load('output_feb17_2/episode_data/0')

In [ ]:
plt.figure(dpi=200)
i1 = 516
i2 = 595
plt.subplot(131)
plt.imshow(x['images'][i1].permute(1, 2, 0))
plt.subplot(132)
plt.imshow(x['images'][i2].permute(1, 2, 0))
plt.subplot(133)
print((x['images'][i2].permute(1, 2, 0) - x['images'][i1].permute(1, 2, 0)).abs().sum())